In [1]:
import uproot
import numpy as np
import ROOT

uproot.default_library = "np"

Welcome to JupyROOT 6.26/06


ldd: exited with unknown exit code (139)


In [2]:
inputfilename = "nanoaod_Y13710.124.al_ttree.root"
outputfilename = "reduced_nanoaod_Y13710.124.al_ttree.root"
correctionfilename = "31535_PID_Performance_Output.root"

file = uproot.open(inputfilename)
tree = file["t"]

In [3]:
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Btag_probAllIP       | RVec<float>              | AsJagged(AsDtype('>f4'), he...
Btag_probNegIP       | RVec<float>              | AsJagged(AsDtype('>f4'), he...
Btag_probPosIP       | RVec<float>              | AsJagged(AsDtype('>f4'), he...
Btag_thrustVector    | ROOT::Math::Displacem... | AsGroup(<TBranchElement 'Bt...
Btag_thrustVector... | ROOT::Math::Cartesian... | AsGroup(<TBranchElement 'fC...
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | ROOT::Math::Cartesian... | AsGroup(<TBranchElement 'Bt...
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | float                    | AsDtype('>f4')
Btag_thrustVector... | floa

In [4]:
GenAlias           = {}
GenAlias["px"]     = "GenPart_vector/GenPart_vector.fCoordinates.fX"
GenAlias["py"]     = "GenPart_vector/GenPart_vector.fCoordinates.fY"
GenAlias["pz"]     = "GenPart_vector/GenPart_vector.fCoordinates.fZ"
GenAlias["E"]      = "GenPart_vector/GenPart_vector.fCoordinates.fT"
GenAlias["mass"]   = "GenPart_mass"
GenAlias["id"]     = "GenPart_pdgId"
GenAlias["parent"] = "GenPart_parentIdx"
GenAlias["status"] = "GenPart_status"

AllSelection = "(status == 1)"
PionSelection = "(status == 1) & ((id == 211) | (id == -211))"
KaonSelection = "(status == 1) & ((id == 321) | (id == -321))"

if "GenPart_vector" in tree:
    Gen = tree.arrays(["px", "py", "pz", "E", "mass", "id", "status", "parent"], aliases = GenAlias)
else:
    class EmptyClass:
        pass
    GenEvent = EmptyClass()
    GenEvent.px = []
    GenEvent.py = []
    GenEvent.pz = []
    GenEvent.E = []
    GenEvent.mass = []
    GenEvent.id = []
    GenEvent.status = []
    GenEvent.parent = []
    Gen = np.array(tree.num_entries * [GenEvent])

SimAlias = {}
SimAlias["px"]     = "SimPart_fourMomentum/SimPart_fourMomentum.fCoordinates.fX"
SimAlias["py"]     = "SimPart_fourMomentum/SimPart_fourMomentum.fCoordinates.fY"
SimAlias["pz"]     = "SimPart_fourMomentum/SimPart_fourMomentum.fCoordinates.fZ"
SimAlias["E"]      = "SimPart_fourMomentum/SimPart_fourMomentum.fCoordinates.fT"
SimAlias["genid"]  = "SimPart_genIdx"
SimAlias["partid"] = 'SimPart_partIdx'
SimAlias["id"]     = 'SimPart_pdgId'
SimAlias["in_id"]  = 'SimPart_originVtxIdx'
SimAlias["out_id"] = 'SimPart_decayVtxIdx'

if "SimPart_genIdx" in tree:
    Sim = tree.arrays(["px", "py", "pz", "E", "genid", "partid", "id", "in_id", "out_id"], aliases = SimAlias)
else:
    class EmptyClass:
        pass
    SimEvent = EmptyClass()
    SimEvent.px = []
    SimEvent.py = []
    SimEvent.pz = []
    SimEvent.E = []
    SimEvent.genid = []
    SimEvent.partid = []
    SimEvent.id = []
    SimEvent.in_id = []
    SimEvent.out_id = []
    Sim = np.array(tree.num_entries * [SimEvent])

RecoAlias           = {}
RecoAlias["px"]     = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fX"
RecoAlias["py"]     = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fY"
RecoAlias["pz"]     = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fZ"
RecoAlias["E"]      = "Part_fourMomentum/Part_fourMomentum.fCoordinates.fT"
RecoAlias["charge"] = "Part_charge"
RecoAlias["id"]     = "Part_pdgId"

Reco = tree.arrays(["px", "py", "pz", "E", "charge", "id"], aliases = RecoAlias)

TrackAlias = {}
TrackAlias["length"] = "Trac_length"
TrackAlias["perigee"] = "Trac_perigee/Trac_perigee.fArray[5]"

if "Trac_length" in tree:
    Track = tree.arrays(["length", "perigee"], aliases = TrackAlias)
else:
    class EmptyClass:
        pass
    TrackEvent = EmptyClass()
    TrackEvent.length = []
    TrackEvent.perigee = [[-999, -999, -999, -999, -999]]
    Track = np.array(tree.num_entries * [TrackEvent])

PIDAlias = {}
PIDAlias["dedx"]         = "Dedx_valueVD"
PIDAlias["ElectronTag"]  = 'Haidc_electronTag'
PIDAlias["HeavyTag"]     = 'Haidc_heavyTag'
PIDAlias["KaonTag"]      = 'Haidc_kaonTag'
PIDAlias["PionTag"]      = 'Haidc_pionTag'
PIDAlias["ProtonTag"]    = 'Haidc_protonTag'
PIDAlias["Selection"]    = 'Haidc_selectionFlag'
PIDAlias["QKaonTag"]     = "Haid_kaonCombined"
PIDAlias["QProtonTag"]   = "Haid_protonCombined"
PIDAlias["MuID"]         = "Muid_tag"
PIDAlias["EleID"]        = "Elid_tag"
PIDAlias["ConversionID"] = "Elid_gammaConversion"

if "Muid_tag" in tree:
    PID = tree.arrays(["dedx", "ElectronTag", "HeavyTag", "PionTag", "KaonTag", "ProtonTag", "Selection", "QKaonTag", "QProtonTag", "MuID", "EleID", "ConversionID"], aliases = PIDAlias)
else:
    PID = tree.arrays(["dedx", "ElectronTag", "HeavyTag", "PionTag", "KaonTag", "ProtonTag", "Selection", "QKaonTag", "QProtonTag", "EleID", "ConversionID"], aliases = PIDAlias)

BTagAlias = {}
BTagAlias["ThrustX"] = "Btag_thrustVector_fCoordinates_fX"
BTagAlias["ThrustY"] = "Btag_thrustVector_fCoordinates_fY"
BTagAlias["ThrustZ"] = "Btag_thrustVector_fCoordinates_fZ"

BTag = tree.arrays(["ThrustX", "ThrustY", "ThrustZ"], aliases = BTagAlias)

EventAlias = {}
EventAlias["Ecm"]   = "Event_cmEnergy"
EventAlias["Nch"]   = 'Event_chargedMult'
EventAlias["Run"]   = "Event_runNumber"
EventAlias["Event"] = "Event_evtNumber"
EventAlias["Fill"]  = "Event_fillNumber"

Event = tree.arrays(["Ecm", "Nch", "Run", "Event", "Fill"], aliases = EventAlias)

In [5]:
EventID = 0

print(len(Reco[EventID].px))
print(len(Reco[EventID].charge))
print(np.count_nonzero(Reco[EventID].charge))
print(len(Track[EventID].length))
print(len(PID[EventID].ElectronTag))
print(len(PID[EventID].MuID))
print(len(PID[EventID].EleID))
print(len(PID[EventID].ConversionID))

print(Reco[EventID].charge)
print(PID[EventID].EleID)
print(PID[EventID].ElectronTag)

3
3
0
3
3
3
3
3
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


In [6]:
def Magnitude(px, py, pz):
    return np.sqrt(px * px + py * py + pz * pz)

def DotProduct(px1, py1, pz1, px2, py2, pz2):
    return px1 * px2 + py1 * py2 + pz1 * pz2

def CosAngle(px1, py1, pz1, px2, py2, pz2):
    return DotProduct(px1, py1, pz1, px2, py2, pz2) / Magnitude(px1, py1, pz1) / Magnitude(px2, py2, pz2)

def GetAngle(px1, py1, pz1, px2, py2, pz2):
    Value = CosAngle(px1, py1, pz1, px2, py2, pz2)
    if Value < -1:
        Value = -1
    if Value > 1:
        Value = 1
    return np.arccos(Value)

In [11]:
correctionfile = ROOT.TFile(correctionfilename)

# Order is pi, K, p
HEffPT = [[None, None, None], [None, None, None], [None, None, None]]
HEffTheta = [[None, None, None], [None, None, None], [None, None, None]]
Norm = [[20808/32327., 1232/32327., 333/32327.], [330/5099., 2282/5099., 356/5099.], [77/1546., 132/1546., 697/1546.]]

HEffPT[0][0] = correctionfile.Get("pT_Efficiencies/Eff_pT_pion_Haidc_as_pion")
HEffPT[0][1] = correctionfile.Get("pT_Efficiencies/Eff_pT_pion_Haidc_as_kaon")
HEffPT[0][2] = correctionfile.Get("pT_Efficiencies/Eff_pT_pion_Haidc_as_proton")
HEffPT[1][0] = correctionfile.Get("pT_Efficiencies/Eff_pT_kaon_Haidc_as_pion")
HEffPT[1][1] = correctionfile.Get("pT_Efficiencies/Eff_pT_kaon_Haidc_as_kaon")
HEffPT[1][2] = correctionfile.Get("pT_Efficiencies/Eff_pT_kaon_Haidc_as_proton")
HEffPT[2][0] = correctionfile.Get("pT_Efficiencies/Eff_pT_proton_Haidc_as_pion")
HEffPT[2][1] = correctionfile.Get("pT_Efficiencies/Eff_pT_proton_Haidc_as_kaon")
HEffPT[2][2] = correctionfile.Get("pT_Efficiencies/Eff_pT_proton_Haidc_as_proton")

HEffTheta[0][0] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_pion_Haidc_as_pion")
HEffTheta[0][1] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_pion_Haidc_as_kaon")
HEffTheta[0][2] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_pion_Haidc_as_proton")
HEffTheta[1][0] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_kaon_Haidc_as_pion")
HEffTheta[1][1] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_kaon_Haidc_as_kaon")
HEffTheta[1][2] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_kaon_Haidc_as_proton")
HEffTheta[2][0] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_proton_Haidc_as_pion")
HEffTheta[2][1] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_proton_Haidc_as_kaon")
HEffTheta[2][2] = correctionfile.Get("Theta_Efficiencies/Eff_Theta_proton_Haidc_as_proton")

def GetEfficiency(PT, CosTheta, Type1, Type2):
    EffPT    = HEffPT[Type1][Type2].GetBinContent(HEffPT[Type1][Type2].FindBin(PT))
    EffTheta = HEffTheta[Type1][Type2].GetBinContent(HEffTheta[Type1][Type2].FindBin(CosTheta))
    return EffPT * EffTheta / Norm[Type1][Type2]


In [12]:
MAX = 10000
outputfile = ROOT.TFile(outputfilename, "RECREATE")
outputtree = ROOT.TTree("Tree", "Strangeness enhancement tree v2")

EventEcm        = np.array([0.0])
EventNch        = np.array([0])
EventRun        = np.array([0])
EventEvent      = np.array([0])
EventFill       = np.array([0])
EventGoodNch    = np.array([0])
EventGoodNneu   = np.array([0])
EventTotalEch   = np.array([0.0])
EventTotalEneu  = np.array([0.0])
EventPassNch    = np.array([0])
EventPassThrust = np.array([0])
EventPassTotalE = np.array([0])
EventPassAll    = np.array([0])

outputtree.Branch("Ecm",        EventEcm,        "Ecm/D")
outputtree.Branch("Nch",        EventNch,        "Nch/L")
outputtree.Branch("Run",        EventRun,        "Run/L")
outputtree.Branch("Event",      EventEvent,      "Event/L")
outputtree.Branch("Fill",       EventFill,       "Fill/L")
outputtree.Branch("GoodNch",    EventGoodNch,    "GoodNch/L")
outputtree.Branch("GoodNneu",   EventGoodNneu,   "GoodNneu/L")
outputtree.Branch("TotalEch",   EventTotalEch,   "TotalEch/D")
outputtree.Branch("TotalEneu",  EventTotalEneu,  "TotalEneu/D")
outputtree.Branch("PassNch",    EventPassNch,    "EventPassNch/L")
outputtree.Branch("PassThrust", EventPassThrust, "EventPassThrust/L")
outputtree.Branch("PassTotalE", EventPassTotalE, "EventPassTotalE/L")
outputtree.Branch("PassAll",    EventPassAll,    "EventPassAll/L")

Thrust = np.array([0.0])
ThrustX = np.array([0.0])
ThrustY = np.array([0.0])
ThrustZ = np.array([0.0])
ThrustTheta = np.array([0.0])

outputtree.Branch("Thrust", Thrust, "Thrust/D")
outputtree.Branch("ThrustX", ThrustX, "ThrustX/D")
outputtree.Branch("ThrustY", ThrustY, "ThrustY/D")
outputtree.Branch("ThrustZ", ThrustZ, "ThrustZ/D")
outputtree.Branch("ThrustTheta", ThrustTheta, "ThrustTheta/D")

NGen          = np.array([0])
GenPx         = np.array(MAX * [0.0])
GenPy         = np.array(MAX * [0.0])
GenPz         = np.array(MAX * [0.0])
GenE          = np.array(MAX * [0.0])
GenM          = np.array(MAX * [0.0])
GenID         = np.array(MAX * [0])
GenStatus     = np.array(MAX * [0])
GenParent     = np.array(MAX * [0])
GenMatchIndex = np.array(MAX * [0])
GenMatchAngle = np.array(MAX * [0.0])

outputtree.Branch("NGen",          NGen,          "NGen/L")
outputtree.Branch("GenPx",         GenPx,         "GenPx[NGen]/D")
outputtree.Branch("GenPy",         GenPy,         "GenPy[NGen]/D")
outputtree.Branch("GenPz",         GenPz,         "GenPz[NGen]/D")
outputtree.Branch("GenE",          GenE,          "GenE[NGen]/D")
outputtree.Branch("GenM",          GenM,          "GenM[NGen]/D")
outputtree.Branch("GenID",         GenID,         "GenID[NGen]/L")
outputtree.Branch("GenStatus",     GenStatus,     "GenStatus[NGen]/L")
outputtree.Branch("GenParent",     GenParent,     "GenParent[NGen]/L")
outputtree.Branch("GenMatchIndex", GenMatchIndex, "GenMatchIndex[NGen]/L")
outputtree.Branch("GenMatchAngle", GenMatchAngle, "GenMatchAngle[NGen]/D")

NReco            = np.array([0])
RecoPx           = np.array(MAX * [0.0])
RecoPy           = np.array(MAX * [0.0])
RecoPz           = np.array(MAX * [0.0])
RecoE            = np.array(MAX * [0.0])
RecoCharge       = np.array(MAX * [0.0])
RecoID           = np.array(MAX * [0])
RecoTrackLength  = np.array(MAX * [0.0])
RecoTrackD0      = np.array(MAX * [0.0])
RecoTrackZ0      = np.array(MAX * [0.0])
RecoPIDElectron  = np.array(MAX * [0])
RecoPIDProton    = np.array(MAX * [0])
RecoPIDPion      = np.array(MAX * [0])
RecoPIDKaon      = np.array(MAX * [0])
RecoPIDHeavy     = np.array(MAX * [0])
RecoPIDQProton   = np.array(MAX * [0.0])
RecoPIDQKaon     = np.array(MAX * [0.0])
RecoMuID         = np.array(MAX * [0])
RecoEleID        = np.array(MAX * [0])
RecoConversionID = np.array(MAX * [0])
RecoGoodTrack    = np.array(MAX * [0])
RecoGoodNeutral  = np.array(MAX * [0])
RecoEfficiencyKAsK   = np.array(MAX * [1.0])
RecoEfficiencyKAsPi  = np.array(MAX * [1.0])
RecoEfficiencyKAsP   = np.array(MAX * [1.0])
RecoEfficiencyPiAsK  = np.array(MAX * [1.0])
RecoEfficiencyPiAsPi = np.array(MAX * [1.0])
RecoEfficiencyPiAsP  = np.array(MAX * [1.0])
RecoEfficiencyPAsK   = np.array(MAX * [1.0])
RecoEfficiencyPAsPi  = np.array(MAX * [1.0])
RecoEfficiencyPAsP   = np.array(MAX * [1.0])

outputtree.Branch("NReco",            NReco,            "NReco/L")
outputtree.Branch("RecoPx",           RecoPx,           "RecoPx[NReco]/D")
outputtree.Branch("RecoPy",           RecoPy,           "RecoPy[NReco]/D")
outputtree.Branch("RecoPz",           RecoPz,           "RecoPz[NReco]/D")
outputtree.Branch("RecoE",            RecoE,            "RecoE[NReco]/D")
outputtree.Branch("RecoCharge",       RecoCharge,       "RecoCharge[NReco]/D")
outputtree.Branch("RecoID",           RecoID,           "RecoID[NReco]/L")
outputtree.Branch("RecoTrackLength",  RecoTrackLength,  "RecoTrackLength[NReco]/D")
outputtree.Branch("RecoTrackD0",      RecoTrackD0,      "RecoTrackD0[NReco]/D")
outputtree.Branch("RecoTrackZ0",      RecoTrackZ0,      "RecoTrackZ0[NReco]/D")
outputtree.Branch("RecoPIDElectron",  RecoPIDElectron,  "RecoPIDElectron[NReco]/L")
outputtree.Branch("RecoPIDProton",    RecoPIDProton,    "RecoPIDProton[NReco]/L")
outputtree.Branch("RecoPIDKaon",      RecoPIDKaon,      "RecoPIDKaon[NReco]/L")
outputtree.Branch("RecoPIDPion",      RecoPIDPion,      "RecoPIDPion[NReco]/L")
outputtree.Branch("RecoPIDHeavy",     RecoPIDHeavy,     "RecoPIDHeavy[NReco]/L")
outputtree.Branch("RecoPIDQProton",   RecoPIDQProton,   "RecoPIDQProton[NReco]/D")
outputtree.Branch("RecoPIDQKaon",     RecoPIDQKaon,     "RecoPIDQKaon[NReco]/D")
outputtree.Branch("RecoMuID",         RecoMuID,         "RecoMuID[NReco]/L")
outputtree.Branch("RecoEleID",        RecoEleID,        "RecoEleID[NReco]/L")
outputtree.Branch("RecoConversionID", RecoConversionID, "RecoConversionID[NReco]/L")
outputtree.Branch("RecoGoodTrack",    RecoGoodTrack,    "RecoGoodTrack[NReco]/L")
outputtree.Branch("RecoGoodNeutral",  RecoGoodNeutral,  "RecoGoodNeutral[NReco]/L")
outputtree.Branch("RecoEfficiencyKAsK",   RecoEfficiencyKAsK,   "RecoEfficiencyKAsK[NReco]/D")
outputtree.Branch("RecoEfficiencyKAsPi",  RecoEfficiencyKAsPi,  "RecoEfficiencyKAsPi[NReco]/D")
outputtree.Branch("RecoEfficiencyKAsP",   RecoEfficiencyKAsP,   "RecoEfficiencyKAsP[NReco]/D")
outputtree.Branch("RecoEfficiencyPiAsK",  RecoEfficiencyPiAsK,  "RecoEfficiencyPiAsK[NReco]/D")
outputtree.Branch("RecoEfficiencyPiAsPi", RecoEfficiencyPiAsPi, "RecoEfficiencyPiAsPi[NReco]/D")
outputtree.Branch("RecoEfficiencyPiAsP",  RecoEfficiencyPiAsP,  "RecoEfficiencyPiAsP[NReco]/D")
outputtree.Branch("RecoEfficiencyPAsK",   RecoEfficiencyPAsK,   "RecoEfficiencyPAsK[NReco]/D")
outputtree.Branch("RecoEfficiencyPAsPi",  RecoEfficiencyPAsPi,  "RecoEfficiencyPAsPi[NReco]/D")
outputtree.Branch("RecoEfficiencyPAsP",   RecoEfficiencyPAsP,   "RecoEfficiencyPAsP[NReco]/D")

NSim     = np.array([0])
SimPx    = np.array(MAX * [0.0])
SimPy    = np.array(MAX * [0.0])
SimPz    = np.array(MAX * [0.0])
SimE     = np.array(MAX * [0.0])
SimID    = np.array(MAX * [0])
SimGenID = np.array(MAX * [0])
SimIn    = np.array(MAX * [0])
SimOut   = np.array(MAX * [0])

outputtree.Branch("NSim",     NSim,     "NSim/L")
outputtree.Branch("SimPx",    SimPx,    "SimPx[NGen]/D")
outputtree.Branch("SimPy",    SimPy,    "SimPy[NGen]/D")
outputtree.Branch("SimPz",    SimPz,    "SimPz[NGen]/D")
outputtree.Branch("SimE",     SimE,     "SimE[NGen]/D")
outputtree.Branch("SimID",    SimID,    "SimID[NGen]/L")
# outputtree.Branch("SimGenID", SimGenID, "SimGenID[NGen]/L")
# outputtree.Branch("SimIn",    SimIn,    "SimIn[NGen]/L")
# outputtree.Branch("SimOut",   SimOut,   "SimOut[NGen]/L")

NKShort          = np.array([0])
KShortPx         = np.array(MAX * [0.0])
KShortPy         = np.array(MAX * [0.0])
KShortPz         = np.array(MAX * [0.0])
KShortE          = np.array(MAX * [0.0])
KShortSim1ID     = np.array(MAX * [0])
KShortSim2ID     = np.array(MAX * [0])
KShortReco1ID    = np.array(MAX * [0])
KShortReco2ID    = np.array(MAX * [0])
KShortReco1Angle = np.array(MAX * [0.0])
KShortReco2Angle = np.array(MAX * [0.0])
KShortRecoPx     = np.array(MAX * [0.0])
KShortRecoPy     = np.array(MAX * [0.0])
KShortRecoPz     = np.array(MAX * [0.0])
KShortRecoE      = np.array(MAX * [0.0])

outputtree.Branch("NKShort",                   NKShort,          "NKShort/L")
outputtree.Branch("KShortPx[NKShort]",         KShortPx,         "KShortPx[NKShort]/D")
outputtree.Branch("KShortPy[NKShort]",         KShortPy,         "KShortPy[NKShort]/D")
outputtree.Branch("KShortPz[NKShort]",         KShortPz,         "KShortPz[NKShort]/D")
outputtree.Branch("KShortE[NKShort]",          KShortE,          "KShortE[NKShort]/D")
outputtree.Branch("KShortSim1ID[NKShort]",     KShortSim1ID,     "KShortSim1ID[NKShort]/L")
outputtree.Branch("KShortSim2ID[NKShort]",     KShortSim2ID,     "KShortSim2ID[NKShort]/L")
outputtree.Branch("KShortReco1ID[NKShort]",    KShortReco1ID,    "KShortReco1ID[NKShort]/L")
outputtree.Branch("KShortReco2ID[NKShort]",    KShortReco2ID,    "KShortReco2ID[NKShort]/L")
outputtree.Branch("KShortReco1Angle[NKShort]", KShortReco1Angle, "KShortReco1Angle[NKShort]/D")
outputtree.Branch("KShortReco2Angle[NKShort]", KShortReco2Angle, "KShortReco2Angle[NKShort]/D")
outputtree.Branch("KShortRecoPx[NKShort]",     KShortRecoPx,     "KShortRecoPx[NKShort]/D")
outputtree.Branch("KShortRecoPy[NKShort]",     KShortRecoPy,     "KShortRecoPy[NKShort]/D")
outputtree.Branch("KShortRecoPz[NKShort]",     KShortRecoPz,     "KShortRecoPz[NKShort]/D")
outputtree.Branch("KShortRecoE[NKShort]",      KShortRecoE,      "KShortRecoE[NKShort]/D")

NPhi          = np.array([0])
PhiPx         = np.array(MAX * [0.0])
PhiPy         = np.array(MAX * [0.0])
PhiPz         = np.array(MAX * [0.0])
PhiE          = np.array(MAX * [0.0])
PhiGen1ID     = np.array(MAX * [0])
PhiGen2ID     = np.array(MAX * [0])
PhiReco1ID    = np.array(MAX * [0])
PhiReco2ID    = np.array(MAX * [0])
PhiReco1Angle = np.array(MAX * [0.0])
PhiReco2Angle = np.array(MAX * [0.0])
PhiRecoPx     = np.array(MAX * [0.0])
PhiRecoPy     = np.array(MAX * [0.0])
PhiRecoPz     = np.array(MAX * [0.0])
PhiRecoE      = np.array(MAX * [0.0])

outputtree.Branch("NPhi",                NPhi,          "NPhi/L")
outputtree.Branch("PhiPx[NPhi]",         PhiPx,         "PhiPx[NPhi]/D")
outputtree.Branch("PhiPy[NPhi]",         PhiPy,         "PhiPy[NPhi]/D")
outputtree.Branch("PhiPz[NPhi]",         PhiPz,         "PhiPz[NPhi]/D")
outputtree.Branch("PhiE[NPhi]",          PhiE,          "PhiE[NPhi]/D")
outputtree.Branch("PhiGen1ID[NPhi]",     PhiGen1ID,     "PhiGen1ID[NPhi]/L")
outputtree.Branch("PhiGen2ID[NPhi]",     PhiGen2ID,     "PhiGen2ID[NPhi]/L")
outputtree.Branch("PhiReco1ID[NPhi]",    PhiReco1ID,    "PhiReco1ID[NPhi]/L")
outputtree.Branch("PhiReco2ID[NPhi]",    PhiReco2ID,    "PhiReco2ID[NPhi]/L")
outputtree.Branch("PhiReco1Angle[NPhi]", PhiReco1Angle, "PhiReco1Angle[NPhi]/D")
outputtree.Branch("PhiReco2Angle[NPhi]", PhiReco2Angle, "PhiReco2Angle[NPhi]/D")
outputtree.Branch("PhiRecoPx[NPhi]",     PhiRecoPx,     "PhiRecoPx[NPhi]/D")
outputtree.Branch("PhiRecoPy[NPhi]",     PhiRecoPy,     "PhiRecoPy[NPhi]/D")
outputtree.Branch("PhiRecoPz[NPhi]",     PhiRecoPz,     "PhiRecoPz[NPhi]/D")
outputtree.Branch("PhiRecoE[NPhi]",      PhiRecoE,      "PhiRecoE[NPhi]/D")

# now we loop over the tree and fill things in
EntryCount = len(Gen)
for iE in range(EntryCount):

    # Event level quantities
    EventEcm[0]   = Event[iE].Ecm
    EventNch[0]   = Event[iE].Nch
    EventRun[0]   = Event[iE].Run
    EventEvent[0] = Event[iE].Event
    EventFill[0]  = Event[iE].Fill

    ThrustX[0]     = BTag[iE].ThrustX
    ThrustY[0]     = BTag[iE].ThrustY
    ThrustZ[0]     = BTag[iE].ThrustZ
    ThrustTheta[0] = np.arccos(BTag[iE].ThrustZ)
    
    # Gen particles
    NGen[0] = len(Gen[iE].px)
    GenPx[0:NGen[0]] = np.array(Gen[iE].px)
    GenPy[0:NGen[0]] = np.array(Gen[iE].py)
    GenPz[0:NGen[0]] = np.array(Gen[iE].pz)
    GenE[0:NGen[0]] = np.array(Gen[iE].E)
    GenM[0:NGen[0]] = np.array(Gen[iE].mass)
    GenID[0:NGen[0]] = np.array(Gen[iE].id)
    GenStatus[0:NGen[0]] = np.array(Gen[iE].status)
    GenParent[0:NGen[0]] = np.array(Gen[iE].parent)

    # Reco particles
    RecoPx[:]     = 0
    RecoPy[:]     = 0
    RecoPz[:]     = 0
    RecoE[:]      = 0
    RecoCharge[:] = 0
    RecoID[:]     = 0
    
    NReco[0] = len(Reco[iE].px)
    RecoPx[0:NReco[0]] = np.array(Reco[iE].px)
    RecoPy[0:NReco[0]] = np.array(Reco[iE].py)
    RecoPz[0:NReco[0]] = np.array(Reco[iE].pz)
    RecoE[0:NReco[0]] = np.array(Reco[iE].E)
    RecoCharge[0:NReco[0]] = np.array(Reco[iE].charge)
    RecoID[0:NReco[0]] = np.array(Reco[iE].id)

    RecoPT = np.sqrt(RecoPx * RecoPx + RecoPy * RecoPy)
    RecoP = np.sqrt(RecoPx * RecoPx + RecoPy * RecoPy + RecoPz * RecoPz)
    RecoTheta = np.arccos(RecoPz / RecoP)

    # Track quantities
    RecoTrackLength[:] = -999
    RecoTrackD0[:]     = -999
    RecoTrackZ0[:]     = -999

    RecoTrackLength[0:NReco[0]] = np.array(Track[iE].length)
    RecoTrackD0[0:NReco[0]]     = np.array(Track[iE].perigee)[:, 1]
    RecoTrackZ0[0:NReco[0]]     = np.array(Track[iE].perigee)[:, 2]

    # Reco particle IDs (only charged particles)
    RecoPIDElectron[:]  = -999
    RecoPIDProton[:]    = -999
    RecoPIDPion[:]      = -999
    RecoPIDKaon[:]      = -999
    RecoPIDHeavy[:]     = -999
    RecoPIDQProton[:]   = -999
    RecoPIDQKaon[:]     = -999
    RecoMuID[:]         = -999
    RecoEleID[:]        = -999
    RecoConversionID[:] = -999

    RecoPIDElectron[0:NReco[0]]  = np.array(PID[iE].ElectronTag)
    RecoPIDProton[0:NReco[0]]    = np.array(PID[iE].ProtonTag)
    RecoPIDPion[0:NReco[0]]      = np.array(PID[iE].PionTag)
    RecoPIDKaon[0:NReco[0]]      = np.array(PID[iE].KaonTag)
    RecoPIDHeavy[0:NReco[0]]     = np.array(PID[iE].HeavyTag)
    RecoPIDQProton[0:NReco[0]]   = np.array(PID[iE].QProtonTag)
    RecoPIDQKaon[0:NReco[0]]     = np.array(PID[iE].QKaonTag)
    RecoMuID[0:NReco[0]]         = np.array(PID[iE].MuID)
    RecoEleID[0:NReco[0]]        = np.array(PID[iE].EleID)
    RecoConversionID[0:NReco[0]] = np.array(PID[iE].ConversionID)

    # Reco particle ID weights
    RecoEfficiencyPiAsPi[0:NReco[0]] = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 0, 0) for x in range(NReco[0])]
    RecoEfficiencyPiAsK[0:NReco[0]]  = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 0, 1) for x in range(NReco[0])]
    RecoEfficiencyPiAsP[0:NReco[0]]  = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 0, 2) for x in range(NReco[0])]
    RecoEfficiencyKAsPi[0:NReco[0]]  = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 1, 0) for x in range(NReco[0])]
    RecoEfficiencyKAsK[0:NReco[0]]   = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 1, 1) for x in range(NReco[0])]
    RecoEfficiencyKAsP[0:NReco[0]]   = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 1, 2) for x in range(NReco[0])]
    RecoEfficiencyPAsPi[0:NReco[0]]  = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 2, 0) for x in range(NReco[0])]
    RecoEfficiencyPAsK[0:NReco[0]]   = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 2, 1) for x in range(NReco[0])]
    RecoEfficiencyPAsP[0:NReco[0]]   = [GetEfficiency(RecoPT[x], RecoPz[x] / RecoP[x], 2, 2) for x in range(NReco[0])]

    # Reco particle selection
    RecoGoodTrack[:] = 0
    for iR in range(NReco[0]):
        if RecoCharge[iR] == 0:
            continue
        if np.abs(RecoTrackD0[iR]) > 4:
            continue
        if np.abs(RecoTrackZ0[iR] * np.sin(RecoTheta[iR])) > 4:
            continue
        if RecoPT[iR] < 0.4:
            continue
        if RecoPT[iR] > 100:
            continue
        if RecoTheta[iR] < np.pi / 9 or RecoTheta[iR] > 8 * np.pi / 9:
            continue
        RecoGoodTrack[iR] = 1

    RecoGoodNeutral[:] = 0
    for iR in range(NReco[0]):
        if RecoCharge[iR] != 0:
            continue
        if RecoE[iR] < 0.5:
            continue
        if RecoE[iR] > 50:
            continue
        if RecoTheta[iR] < np.pi / 9 or RecoTheta[iR] > 8 * np.pi / 9:
            continue
        RecoGoodNeutral[iR] = 1
    
    # Final stuff for events
    EventGoodNch[0]   = np.sum(RecoGoodTrack == 1)
    EventGoodNneu[0]  = np.sum(RecoGoodNeutral == 1)
    EventTotalEch[0]  = np.sum(RecoE[RecoGoodTrack==1])
    EventTotalEneu[0] = np.sum(RecoE[RecoGoodNeutral==1])
    
    EventPassNch[0]    = EventGoodNch[0] >= 7
    EventPassThrust[0] = (ThrustTheta[0] > np.pi / 6 and ThrustTheta[0] < 5 * np.pi / 6)
    EventPassTotalE[0] = (EventTotalEch[0] + EventTotalEneu[0]) > EventEcm[0] * 0.5
    EventPassAll[0]    = EventPassNch[0] and EventPassThrust[0] and EventPassTotalE[0]

    # Gen-reco match
    GenMatchIndex[:] = -999
    GenMatchAngle[:] = -999
    
    for iG in range(NGen[0]):
        BestIndex = -1
        BestDistance = -1
        for iR in range(NReco[0]):
            Distance = GetAngle(GenPx[iG], GenPy[iG], GenPz[iG], RecoPx[iR], RecoPy[iR], RecoPz[iR])

            if BestIndex < 0 or BestDistance > Distance:
                BestIndex = iR
                BestDistance = Distance

        GenMatchIndex[iG] = BestIndex
        GenMatchAngle[iG] = BestDistance

    # Sim tracks
    NSim[0] = len(Sim[iE].px)
    SimPx[0:NSim[0]]    = np.array(Sim[iE].px)
    SimPy[0:NSim[0]]    = np.array(Sim[iE].py)
    SimPz[0:NSim[0]]    = np.array(Sim[iE].pz)
    SimE[0:NSim[0]]     = np.array(Sim[iE].E)
    SimID[0:NSim[0]]    = np.array(Sim[iE].id)
    SimGenID[0:NSim[0]] = np.array(Sim[iE].genid)
    SimIn[0:NSim[0]]    = np.array(Sim[iE].in_id)
    SimOut[0:NSim[0]]   = np.array(Sim[iE].out_id)

    # KShort's
    NKShort[0] = 0
    for iG in [x for x in range(NGen[0]) if GenStatus[x] == 4 and GenID[x] == 310]:
        SimIndex = [x for x in range(NSim[0]) if iG == SimGenID[x]]
        DecayIndex = [x for x in range(NSim[0]) if SimOut[SimIndex[0]] == SimIn[x]]

        Conserve =              np.abs(GenPx[iG] - np.sum(SimPx[DecayIndex])) < 0.01
        Conserve = Conserve and np.abs(GenPy[iG] - np.sum(SimPy[DecayIndex])) < 0.01
        Conserve = Conserve and np.abs(GenPz[iG] - np.sum(SimPz[DecayIndex])) < 0.01
        Conserve = Conserve and np.abs(GenE[iG]  - np.sum(SimE[DecayIndex]))  < 0.01

        # 41 and -41 are pi+ and pi- in internal Delphi
        if Conserve == True and 41 in SimID[DecayIndex] and -41 in SimID[DecayIndex]:
            KShortPx[NKShort[0]] = GenPx[iG]
            KShortPy[NKShort[0]] = GenPy[iG]
            KShortPz[NKShort[0]] = GenPz[iG]
            KShortE[NKShort[0]]  = GenE[iG]

            KShortSim1ID[NKShort[0]] = [x for x in DecayIndex if SimID[x] == 41][0]
            KShortSim2ID[NKShort[0]] = [x for x in DecayIndex if SimID[x] == -41][0]

            KShortReco1ID[NKShort[0]] = -999
            KShortReco1Angle[NKShort[0]] = -999.
            for iR in range(NReco[0]):
                Distance = GetAngle(SimPx[KShortSim1ID[NKShort[0]]], SimPy[KShortSim1ID[NKShort[0]]], SimPz[KShortSim1ID[NKShort[0]]], RecoPx[iR], RecoPy[iR], RecoPz[iR])
                if KShortReco1ID[NKShort[0]] < 0 or KShortReco1Angle[NKShort[0]] > Distance:
                    KShortReco1ID[NKShort[0]] = iR
                    KShortReco1Angle[NKShort[0]] = Distance

            KShortReco2ID[NKShort[0]] = -999
            KShortReco2Angle[NKShort[0]] = -999.
            for iR in range(NReco[0]):
                Distance = GetAngle(SimPx[KShortSim2ID[NKShort[0]]], SimPy[KShortSim2ID[NKShort[0]]], SimPz[KShortSim2ID[NKShort[0]]], RecoPx[iR], RecoPy[iR], RecoPz[iR])
                if KShortReco2ID[NKShort[0]] < 0 or KShortReco2Angle[NKShort[0]] > Distance:
                    KShortReco2ID[NKShort[0]] = iR
                    KShortReco2Angle[NKShort[0]] = Distance

            if KShortReco1ID[NKShort[0]] >= 0 and KShortReco2ID[NKShort[0]] >= 0:
                KShortRecoPx[NKShort[0]] = RecoPx[KShortReco1ID[NKShort[0]]] + RecoPx[KShortReco2ID[NKShort[0]]]
                KShortRecoPy[NKShort[0]] = RecoPy[KShortReco1ID[NKShort[0]]] + RecoPy[KShortReco2ID[NKShort[0]]]
                KShortRecoPz[NKShort[0]] = RecoPz[KShortReco1ID[NKShort[0]]] + RecoPz[KShortReco2ID[NKShort[0]]]
                RecoE1 = np.sqrt(RecoPx[KShortReco1ID[NKShort[0]]] * RecoPx[KShortReco1ID[NKShort[0]]] + RecoPy[KShortReco1ID[NKShort[0]]] * RecoPy[KShortReco1ID[NKShort[0]]] + RecoPz[KShortReco1ID[NKShort[0]]] * RecoPz[KShortReco1ID[NKShort[0]]] + 0.13957 * 0.13957)
                RecoE2 = np.sqrt(RecoPx[KShortReco2ID[NKShort[0]]] * RecoPx[KShortReco2ID[NKShort[0]]] + RecoPy[KShortReco2ID[NKShort[0]]] * RecoPy[KShortReco2ID[NKShort[0]]] + RecoPz[KShortReco2ID[NKShort[0]]] * RecoPz[KShortReco2ID[NKShort[0]]] + 0.13957 * 0.13957)
                KShortRecoE[NKShort[0]]  = RecoE1 + RecoE2
            else:
                KShortRecoPx[NKShort[0]] = -999
                KShortRecoPy[NKShort[0]] = -999
                KShortRecoPz[NKShort[0]] = -999
                KShortRecoE[NKShort[0]] = -999
                
            NKShort[0] = NKShort[0] + 1

    # Phi's
    NPhi[0] = 0
    for iG in [x for x in range(NGen[0]) if GenID[x] == 333]:
        DecayIndex = [x for x in range(NGen[0]) if GenParent[x] == iG]

        # Check Kaons
        if 321 in GenID[DecayIndex] and -321 in GenID[DecayIndex]:
            PhiPx[NPhi[0]] = GenPx[iG]
            PhiPy[NPhi[0]] = GenPy[iG]
            PhiPz[NPhi[0]] = GenPz[iG]
            PhiE[NPhi[0]]  = GenE[iG]

            PhiGen1ID[NPhi[0]] = [x for x in DecayIndex if GenID[x] == 321][0]
            PhiGen2ID[NPhi[0]] = [x for x in DecayIndex if GenID[x] == -321][0]

            PhiReco1ID[NPhi[0]] = -999
            PhiReco1Angle[NPhi[0]] = -999.
            for iR in range(NReco[0]):
                Distance = GetAngle(GenPx[PhiGen1ID[NPhi[0]]], GenPy[PhiGen1ID[NPhi[0]]], GenPz[PhiGen1ID[NPhi[0]]], RecoPx[iR], RecoPy[iR], RecoPz[iR])
                if PhiReco1ID[NPhi[0]] < 0 or PhiReco1Angle[NPhi[0]] > Distance:
                    PhiReco1ID[NPhi[0]] = iR
                    PhiReco1Angle[NPhi[0]] = Distance

            PhiReco2ID[NPhi[0]] = -999
            PhiReco2Angle[NPhi[0]] = -999.
            for iR in range(NReco[0]):
                Distance = GetAngle(GenPx[PhiGen2ID[NPhi[0]]], GenPy[PhiGen2ID[NPhi[0]]], GenPz[PhiGen2ID[NPhi[0]]], RecoPx[iR], RecoPy[iR], RecoPz[iR])
                if PhiReco2ID[NPhi[0]] < 0 or PhiReco2Angle[NPhi[0]] > Distance:
                    PhiReco2ID[NPhi[0]] = iR
                    PhiReco2Angle[NPhi[0]] = Distance

            if PhiReco1ID[NPhi[0]] >= 0 and PhiReco2ID[NPhi[0]] >= 0:
                PhiRecoPx[NPhi[0]] = RecoPx[PhiReco1ID[NPhi[0]]] + RecoPx[PhiReco2ID[NPhi[0]]]
                PhiRecoPy[NPhi[0]] = RecoPy[PhiReco1ID[NPhi[0]]] + RecoPy[PhiReco2ID[NPhi[0]]]
                PhiRecoPz[NPhi[0]] = RecoPz[PhiReco1ID[NPhi[0]]] + RecoPz[PhiReco2ID[NPhi[0]]]
                RecoE1 = np.sqrt(RecoPx[PhiReco1ID[NPhi[0]]] * RecoPx[PhiReco1ID[NPhi[0]]] + RecoPy[PhiReco1ID[NPhi[0]]] * RecoPy[PhiReco1ID[NPhi[0]]] + RecoPz[PhiReco1ID[NPhi[0]]] * RecoPz[PhiReco1ID[NPhi[0]]] + 0.49368 * 0.49368)
                RecoE2 = np.sqrt(RecoPx[PhiReco2ID[NPhi[0]]] * RecoPx[PhiReco2ID[NPhi[0]]] + RecoPy[PhiReco2ID[NPhi[0]]] * RecoPy[PhiReco2ID[NPhi[0]]] + RecoPz[PhiReco2ID[NPhi[0]]] * RecoPz[PhiReco2ID[NPhi[0]]] + 0.49368 * 0.49368)
                PhiRecoE[NPhi[0]]  = RecoE1 + RecoE2
            else:
                PhiRecoPx[NPhi[0]] = -999
                PhiRecoPy[NPhi[0]] = -999
                PhiRecoPz[NPhi[0]] = -999
                PhiRecoE[NPhi[0]] = -999

            NPhi[0] = NPhi[0] + 1

    if EventGoodNch[0] > 2:
        outputtree.Fill()

outputtree.Write()
outputfile.Close()

/tmp/ipykernel_84250/4265719261.py:262: RuntimeWarning: invalid value encountered in divide
  RecoTheta = np.arccos(RecoPz / RecoP)


In [13]:
print("Yay")

Yay
